In [1]:
import time 
import requests, bs4
import re
import sys
import os
import datetime
from urllib.parse import urlparse

import pandas as pd
import numpy as np
from omegaconf import OmegaConf

sys.path.append("../")
from src.scraping import ScrapingBase, ScrapingSponavi

%matplotlib inline
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 出力先
conf_dir = "../config"
conf_cli = OmegaConf.from_cli()
conf_exec = OmegaConf.load(os.path.join(conf_dir, "config_exec.yaml"))
conf_path = OmegaConf.load(os.path.join(conf_dir, "config_path.yaml"))
conf_url = OmegaConf.load(os.path.join(conf_dir, "config_url.yaml"))
conf_team = OmegaConf.load(os.path.join(conf_dir, "config_team.yaml"))
config = OmegaConf.merge(conf_cli, conf_exec, conf_path, conf_url, conf_team)

In [2]:
ss = ScrapingSponavi(config=config,start_date="2021-03-21", end_date="2021-03-21")

In [47]:
url = "https://baseball.yahoo.co.jp/npb/game/2021000142/score?index=0520200"
html = ss.get_html(url)
soup = bs4.BeautifulSoup(html, "html.parser")

get html  https://baseball.yahoo.co.jp/npb/game/2021000142/score?index=0520200
success


In [48]:
result = {}
inning_text = soup.select_one("div[id='live']").select_one("em").get_text(strip=True)

result["index"] = soup.select_one("a[class='bb-gameScoreTable__score']").get("index")
result["inning"] = inning_text[0]
result["top_buttom"] = "top" if inning_text[-1] == "表" else "bottom"
soup_batter = soup.select_one("table[id='batt']")
result["batter_id"] = ss.make_id(soup_batter.select_one("a").get("href").split("/")[-2])
result["batter_side"] = soup_batter.select_one("td.dominantHand").get_text()
soup_pitcher = soup.select_one("div[id='pitcherL']")
result["pitcher_id"] = ss.make_id(soup_pitcher.select_one("a").get("href").split("/")[-2])
result["pitcher_side"] = soup_pitcher.select_one("td.dominantHand").get_text()
soup_result = soup.select_one("div[id='result']")
result["result_main"] = soup_result.select_one("span").get_text()
result["result_sub"] = soup_result.select_one("em").get_text()

for i in range(1, 4):
    base = soup_result.select_one(f"div[id='base{i}']")
    result[f"base_{i}"] = base.get("href") if base is not None else np.nan

if result["result_main"] == "試合終了":
    print(1)
next_index = soup.select_one("a[id='btn_next']").get("index")
result

AttributeError: 'NoneType' object has no attribute 'select_one'

In [10]:
datetime_now = datetime.datetime.now()
str_datetime_now = datetime_now.strftime("%Y-%m-%d %H:%m:%S")
str_date_now = datetime_now.strftime("%Y-%m-%d")

pb_params = [("p", "pitcher"), ("b", "batter")]
for pb in pb_params:
    df_all = pd.DataFrame() 
    for team in ss.team_list:
        target_url = ss.base_url + "/teams/" + str(ss.team_dict[team].id) + "/memberlist?kind=" + pb[0]
        html = ss.get_html(target_url)
        soup = bs4.BeautifulSoup(html, "html.parser")
        tb = soup.find_all('table')[0] 
        df = pd.read_html(str(tb),encoding='utf-8', header=0)[0]
        df = df.drop(len(df)-1) # 最終行を削除
        df['player_id'] = [ss.get_id(row.select_one("a").get("href").split("/")[-2]) for row in tb.select("tr[class='bb-playerTable__row']")]
        df[df == "-"] = np.nan
        df["team_id"] = str(ss.team_dict[team].id)
        df["team_name"] = team
        df_all = df_all.append(df)

        df_all["exec_date"] = str_date_now
        df_all["exec_datetime"] = str_datetime_now

        ss.save_csv(df_all, file_path="../data/tsv/player_result/lake_player_result_{}.tsv".format(pb[1]))

get html  https://baseball.yahoo.co.jp/npb/teams/1/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/3/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/4/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/5/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/6/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/2/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/12/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/9/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/7/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/376/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/8/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/11/memberlist?kind=p
success
get html  https://baseball.yahoo.co.jp/npb/teams/1/memberlist?kind=b
suc

In [7]:
df_all

,背番号,選手名,打 率,試 合,打 席,打 数,安 打,二塁打,三塁打,本塁打,...,出塁率,長打率,ＯＰＳ,得点圏,失 策,player_id,team_id,team_name,exec_date,exec_datetime
0,00,湯浅 大,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb1700049,1,巨人,2021-03-27,2021-03-27 14:03:36
1,0,増田 大輝,.000,2,1,1,0,0,0,0,...,.000,.000,.000,NaN,1,npb1500054,1,巨人,2021-03-27,2021-03-27 14:03:36
2,2,陽 岱鋼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb600037,1,巨人,2021-03-27,2021-03-27 14:03:36
3,5,中島 宏之,.000,1,1,1,0,0,0,0,...,.000,.000,.000,NaN,0,npb12000,1,巨人,2021-03-27,2021-03-27 14:03:36
4,6,坂本 勇人,.429,2,9,7,3,0,0,0,...,.556,.429,.984,1.000,0,npb700003,1,巨人,2021-03-27,2021-03-27 14:03:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,120,岡﨑 大輔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb1600152,11,オリックス,2021-03-27,2021-03-27 14:03:36
37,123,稲富 宏樹,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb1700117,11,オリックス,2021-03-27,2021-03-27 14:03:36
38,125,西浦 颯大,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb1700114,11,オリックス,2021-03-27,2021-03-27 14:03:36
39,127,田城 飛翔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,npb1600170,11,オリックス,2021-03-27,2021-03-27 14:03:36


In [ ]:
ss.save_csv(df_all, file_path="../data/player_result/lake_player_result")

In [16]:
df_all

""


In [32]:
soup = bs4.BeautifulSoup(html, "html.parser")
tb = soup.find_all('table')[0] 
df = pd.read_html(str(tb),encoding='utf-8', header=0)[0]
df = df.drop(len(df)-1) # 最終行を削除
df['player_id'] = [ss.get_id(row.select_one("a").get("href").split("/")[-2]) for row in tb.select("tr[class='bb-playerTable__row']")]
df[df == "-"] = np.nan

,背番号,選手名,防御率,登 板,先 発,完 投,完 封,Ｑ Ｓ,勝 利,敗 戦,...,与四球,与死球,暴 投,ボーク,失 点,自責点,被打率,Ｋ／ＢＢ,ＷＨＩＰ,player_id
0,11,平内 龍太,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb2000005
1,12,デラロサ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1900012
2,15,サンチェス,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1900143
3,17,大竹 寛,0.00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,.000,NaN,0.00,npb12179
4,18,菅野 智之,4.50,1,1,0,0,1,0,0,...,3,0,0,0,3,3,.333,1.33,1.83,npb1200041
5,19,山崎 伊織,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb2000006
6,20,戸郷 翔征,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1800028
7,21,井納 翔一,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1200050
8,23,野上 亮磨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb900055
9,26,今村 信貴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1100038


,背番号,選手名,防御率,登 板,先 発,完 投,完 封,Ｑ Ｓ,勝 利,敗 戦,...,奪三振率,与四球,与死球,暴 投,ボーク,失 点,自責点,被打率,Ｋ／ＢＢ,ＷＨＩＰ
0,11,平内 龍太,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,12,デラロサ,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,15,サンチェス,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,17,大竹 寛,0.00,1,0,0,0,0,0,0,...,0.00,0,0,0,0,0,0,.000,-,0.00
4,18,菅野 智之,4.50,1,1,0,0,1,0,0,...,6.00,3,0,0,0,3,3,.333,1.33,1.83
5,19,山崎 伊織,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,20,戸郷 翔征,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,21,井納 翔一,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,23,野上 亮磨,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,26,今村 信貴,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


['npb2000005',
 'npb1900012',
 'npb1900143',
 'npb12179',
 'npb1200041',
 'npb2000006',
 'npb1800028',
 'npb1200050',
 'npb900055',
 'npb1100038',
 'npb1200089',
 'npb1900017',
 'npb1500046',
 'npb1500052',
 'npb1700002',
 'npb1600060',
 'npb1800023',
 'npb1900136',
 'npb1400102',
 'npb1600100',
 'npb2000008',
 'npb1100040',
 'npb2000010',
 'npb1500113',
 'npb1800026',
 'npb1300087',
 'npb1600064',
 'npb1900019',
 'npb1800031',
 'npb1600072',
 'npb1600068',
 'npb2000014',
 'npb1800030',
 'npb2000020',
 'npb2000015',
 'npb1700053',
 'npb2000018',
 'npb1700042',
 'npb2000021',
 'npb1900016',
 'npb1600061',
 'npb1500048',
 'npb2000019',
 'npb1800025']

In [11]:
96-45 

51

In [5]:
dfs

[    èçªå·         é¸æå  é²å¾¡ç  ç»ãæ¿  å
 ãçº  å®ãæ  \
 0          11      å¹³å é¾å¤ª          -          -          -          -   
 1          12      ãã©ã­ãµ          -          -          -          -   
 2          15   ãµã³ãã§ã¹          -          -          -          -   
 3          17        å¤§ç«¹ å¯       0.00          1          0          0   
 4          18     è
 é æºä¹       4.50          1          1          0   
 5          19     å±±å´ ä¼ç¹          -          -          -          -   
 6          20     æ¸é· ç¿å¾          -          -          -          -   
 7          21     äºç´ ç¿ä¸          -          -          -          -   
 8          23     éä¸ äº®ç£¨          -          -          -          -   
 9          26     ä»æ ä¿¡è²´          -          -          -          -   
 10         30     éµè°· é½å¹³       0.00          1          0          0   
 11         33        å¤ªç° é¾      

In [ ]:
url_schedule_day = 'https://baseball.yahoo.co.jp/npb/schedule/

In [20]:
import re

content = "https://baseball.yahoo.co.jp/npb/game/2021000011/index"
result = re.search("\d+", content)
if result:
    print(result.group())

2021000011


In [8]:
print(url)
index = re.search(r'index=\d+', url)
res = requests.get(url)

try:
    res.raise_for_status() 
except requests.exceptions.RequestException:
    print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
    #break

soup = bs4.BeautifulSoup(res.text, "html.parser")
elems = soup.select('.bb-calendarTable__status')
elems2 = soup.select('.bb-calendarTable__venue')

#print(elems)
for elem,elem2 in zip(elems,elems2):
    url = elem.get('href')
    date = url[38:46]
    venue = elem2.text
    #print(venue)
    game_yokohama.loc[len(game_yokohama)] = [url,date,venue]
time.sleep(1)

NameError: name 'url' is not defined